# NEW notebook
Works for whole cycle of simulation

In [156]:
%matplotlib nbagg

import numpy as np
import strid
import matplotlib.pyplot as plt
import scipy.signal
import functions as fun
import clustering

In [157]:
#Perform covSSI on generated data to get the estimated modes
stochastic_data_path = "results/data-stochastic-8-floor.npz"
combined_data_path = "results/data-combined_8_floor.npz"
ssid, modes = fun.covSSI(stochastic_data_path)

In [158]:
%matplotlib nbagg
stabdiag = strid.StabilizationDiagram()
modes = stabdiag.plot(modes)

f, psd = ssid.psdy(nperseg=2**10)
stabdiag.axes_psd.semilogy(f, np.trace(np.abs(psd)), color=(0., 0., 0., .5), lw=.3)
plt.savefig("figures/cov_SSI_output.jpg", dpi=300)

<IPython.core.display.Javascript object>

In [297]:
#Initialize a clustering object with the given parameters
cluster = clustering.Cluster(modes,
                             ssid,
                             linkage='average',
                             d_c=0.02,
                             distance_matrix='mac',
                             power=0.13,
                             neighbour=True
                             )

In [298]:
#Run the clustering on the generated data and save results
cluster.run_clustering()

Number of physical modes = 350
Distance matrix computational time = 1.1930959224700928sec
Check integrity of the distane matrix:
Max value = 0.9999998057459687
Mean value = 0.8698991065303585
Min value = 0.0
37
Number of clusters identified: 38
Clustering algorithm computational time = 1.3861138820648193 sec


In [299]:
%matplotlib nbagg
#Making a new stabilization diagram with the physical modes
stabdiag = strid.StabilizationDiagram()
stabdiag.plot_hierarchies(cluster.structural_modes_dict)

f, psd = ssid.psdy(nperseg=2**10)

stabdiag.axes_psd.semilogy(f, np.trace(np.abs(psd)), color=(0., 0., 0., .5), lw=.3)
plt.savefig("figures/clustered_stabdiag_avg.jpg", dpi=300)

<IPython.core.display.Javascript object>

In [300]:
#Extract modal features
est_frequencies, est_damping, est_modeshapes = cluster.extract_modal_features(stabdiag)

#est_frequencies = est_frequencies[:-1]

data = np.load("results/data-stochastic-8-floor.npz")

true_f = data["true_frequencies"]
true_xi = data["true_damping"]
true_modeshapes = data["true_modeshapes"]

#Want to measure model accuracy on difference in frequency
eps_freq = np.average(np.abs(true_f-est_frequencies))
print("Epsilon frequency = %.4f" %eps_freq)

Epsilon frequency = 0.0339


/Users/emilneby/PyCharmProjects/TKT4550_Prosjekt/src/clustering.py:173: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  sorted_features_list = np.array(list(sorted_features_dict.values()))


In [291]:
print(est_modeshapes)

[array([ 0.0011831 +0.01996592j, -0.00405328+0.03681565j,
        -0.00406428+0.04931858j, -0.01441917+0.06878751j,
        -0.01012428+0.08337107j, -0.01630645+0.08440667j,
        -0.0102961 +0.0988222j , -0.01935364+0.10556655j])
 array([-0.00673781+0.02867324j, -0.01294452+0.04802217j,
        -0.01364059+0.05509876j, -0.01209512+0.04399937j,
        -0.00451941+0.02094275j,  0.00272291-0.01058962j,
         0.00918491-0.03643091j,  0.01275312-0.05220362j])
 array([ 5.01832464e-05+2.08794281e-04j, -1.60243366e-04+1.15436264e-04j,
         1.25882201e-04-2.03187235e-04j,  6.16399404e-05+1.09193436e-04j,
        -1.06639427e-04+5.51898126e-05j,  1.18451422e-04+2.04311400e-05j,
        -1.73416190e-04-1.03272361e-04j,  7.82327807e-05-9.55736773e-05j])
 array([-0.01901669+0.03812989j, -0.01007004+0.02028767j,
         0.01333263-0.02606959j,  0.01780697-0.03524534j,
        -0.00421072+0.00791956j, -0.01937659+0.03967776j,
        -0.00763029+0.01405125j,  0.01528281-0.03163402j])
 arr

In [292]:
fig = plt.figure("Damping estimate")
axd = fig.add_axes((0.1, 0.1, .8, .8))
axd.set(xlabel='Frequency', ylabel='Damping ratio', title='Estimated and true frequency and damping',
        ylim=(0, .10)
       )
figmodes, axes = plt.subplots(ncols=4, nrows=2, dpi=144)
res = []
for n in range(true_f.size):
    ax = axes.flatten()[n]
    un = true_modeshapes[n]
    fn = true_f[n]
    xin = true_xi[n]
    #nmax = np.argmax([strid.modal_assurance_criterion(mode.v, un) for mode in modes])
    #mode = modes[nmax]

    line, = axd.plot(est_frequencies[n], est_damping[n], 'x')
    line, = axd.plot(fn, xin, 'o', mec=line.get_color(), mfc=(0, 0, 0, 0))


    ferr = (est_frequencies[n]-fn)/fn * 100
    xierr = (est_damping[n]-xin)/xin*100
    mac = strid.modal_assurance_criterion(un, est_modeshapes[n])
    res.append([n, ferr, xierr, mac*100,])


    v_true = np.r_[0., un]
    v = np.r_[0, est_modeshapes[n]]
    v = strid.modal_scale_factor(v, v_true)*v
    z = np.arange(v.size)
    ax.plot(v_true, z, label='True')
    ax.plot(v.real, z, label='Estimated')

    if n == 2:
        ax.legend(bbox_to_anchor=(.5, 1.20), loc='lower center', ncol=2)
        axd.legend(['Estimated', 'True'],)
    ax.axvline(0., color=(0, 0, 0, .3))

    ax.set_title(f"Mode {n}")
    ax.axis('off')
    ax.set_xlim(-.5, .5)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>